# Unit test

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast


In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
#metrics = [add_y_train_argument(metric) for metric in metrics]
from skforecast.datasets import fetch_dataset



In [7]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
data_test = data.loc[end_train:, :].copy()
levels = ["item_1", "item_2", "item_3"]
exog_features = 'day_of_week'

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
steps = 1
initial_train_size = 100
param_grid = {
    "alpha": np.logspace(-3, 3, 1),
}
lags_grid = [5]
param_grid = list(ParameterGrid(param_grid))

forecaster = ForecasterAutoregMultiSeries(
    regressor=Ridge(random_state=678),
    lags=5,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

# forecaster = ForecasterAutoregMultiVariate(
#     regressor=Ridge(random_state=678),
#     lags=3,
#     steps=1,
#     level="item_1",
#     transformer_series=StandardScaler(),
#     transformer_exog=StandardScaler(),
# )

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=data.loc[:, levels],
        exog=data.loc[:, exog_features] if exog_features else None,
        initial_train_size=initial_train_size,
    )
)

display(data)
display(X_train)
display(X_test)
display(y_train)
display(y_test)
display(X_train_encoding)
display(X_test_encoding)


results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = True
    )

results_one_step_ahead

,item_1,item_2,item_3,day_of_week
date,,,,
2012-01-01,8.253175,21.047727,19.429739,6
2012-01-02,22.777826,26.578125,28.009863,0
2012-01-03,27.549099,31.751042,32.078922,1
2012-01-04,25.895533,24.567708,27.252276,2
2012-01-05,21.379238,18.191667,20.357737,3
...,...,...,...,...
2014-12-28,17.329233,18.189583,20.586030,6
2014-12-29,19.611623,24.539583,28.127390,0
2014-12-30,18.857026,17.677083,21.555782,1


,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast,day_of_week
date,,,,,,,
2012-01-06,-0.419583,1.158317,1.736038,0.069055,-5.005556,0,0.479870
2012-01-07,-0.514822,-0.419583,1.158317,1.736038,0.069055,0,0.975388
2012-01-08,-0.714936,-0.514822,-0.419583,1.158317,1.736038,0,1.470906
2012-01-09,-0.877238,-0.714936,-0.514822,-0.419583,1.158317,0,-1.502202
2012-01-10,-0.899307,-0.877238,-0.714936,-0.514822,-0.419583,0,-1.006684
...,...,...,...,...,...,...,...
2012-04-05,-0.338919,-0.047092,-1.404941,-0.283523,0.769786,2,-0.015648
2012-04-06,-0.356970,-0.338919,-0.047092,-1.404941,-0.283523,2,0.479870
2012-04-07,0.121229,-0.356970,-0.338919,-0.047092,-1.404941,2,0.975388


,lag_1,lag_2,lag_3,lag_4,lag_5,_level_skforecast,day_of_week
date,,,,,,,
2012-04-10,-1.600750,-1.602861,-1.378975,-0.306645,0.130344,0,-1.006684
2012-04-11,-0.666965,-1.600750,-1.602861,-1.378975,-0.306645,0,-0.511166
2012-04-12,0.154228,-0.666965,-1.600750,-1.602861,-1.378975,0,-0.015648
2012-04-13,0.035007,0.154228,-0.666965,-1.600750,-1.602861,0,0.479870
2012-04-14,-0.048676,0.035007,0.154228,-0.666965,-1.600750,0,0.975388
...,...,...,...,...,...,...,...
2014-12-28,0.082209,-0.455532,-1.455941,0.160859,0.870259,2,1.470906
2014-12-29,-0.186787,0.082209,-0.455532,-1.455941,0.160859,2,-1.502202
2014-12-30,1.573370,-0.186787,0.082209,-0.455532,-1.455941,2,-1.006684


date
2012-01-06   -0.514822
2012-01-07   -0.714936
2012-01-08   -0.877238
2012-01-09   -0.899307
2012-01-10   -0.335401
                ...   
2012-04-05   -0.356970
2012-04-06    0.121229
2012-04-07   -0.422595
2012-04-08   -0.144628
2012-04-09   -1.310729
Name: y, Length: 285, dtype: float64

date
2012-04-10   -0.666965
2012-04-11    0.154228
2012-04-12    0.035007
2012-04-13   -0.048676
2012-04-14   -0.464676
                ...   
2014-12-28   -0.186787
2014-12-29    1.573370
2014-12-30    0.039553
2014-12-31   -0.649055
2015-01-01   -0.336703
Name: y, Length: 2991, dtype: float64

...

date
2012-04-10    item_1
2012-04-11    item_1
2012-04-12    item_1
2012-04-13    item_1
2012-04-14    item_1
               ...  
2014-12-28    item_3
2014-12-29    item_3
2014-12-30    item_3
2014-12-31    item_3
2015-01-01    item_3
Length: 2991, dtype: object

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

,levels,lags,lags_label,params,mean_absolute_error__weighted_average,mean_absolute_error__average,mean_absolute_error__pooling,mean_squared_error__weighted_average,mean_squared_error__average,mean_squared_error__pooling,mean_absolute_percentage_error__weighted_average,mean_absolute_percentage_error__average,mean_absolute_percentage_error__pooling,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__average,mean_absolute_scaled_error__pooling,root_mean_squared_scaled_error__weighted_average,root_mean_squared_scaled_error__average,root_mean_squared_scaled_error__pooling,alpha
0,"[item_1, item_2, item_3]","[1, 2, 3, 4, 5]","[1, 2, 3, 4, 5]",{'alpha': 0.001},2.89442,2.89442,2.89442,14.153054,14.153054,14.153054,0.203737,0.203737,0.203737,1.08243,1.08243,1.094723,1.006662,1.006662,1.016628,0.001


In [8]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
levels = ["item_1", "item_2", "item_3"]
exog_features = ['day_of_week']

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 100
    param_grid = {
        "alpha": np.logspace(-1, 1, 1),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:393: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [10]:
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog

series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 20
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )


    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='l1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:393: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1810: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [4]:
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 213
    param_grid = {
        "n_estimators": [10],
        "max_depth": [2],
    }
    lags_grid = [1]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    display(results_backtesting)
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )
    display(results_one_step_ahead)

    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = LGBMRegressor(random_state=678, verbose=-1)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3, forecaster_id=1),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
        forecaster_id=2
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        forecaster_id=3
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        forecaster_id=4
    )
]

for forecaster in forecasters[:1]:
    print(forecaster.forecaster_id)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib'

In [3]:
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)
metrics, predictions = backtesting_forecaster_multiseries(
    forecaster=forecaster,
    series=series_dict,
    exog=exog_dict,
    steps=1,
    initial_train_size=213,
    metric=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric=True,
    n_jobs='auto',
    verbose=False,
    show_progress=True
)

display(metrics)

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)
(
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_encoding,
    X_test_encoding
) = forecaster._train_test_split_one_step_ahead(
        series             = series_dict,
        exog               = exog_dict,
        initial_train_size = 213,
    )

print(y_train)

metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
    forecaster=forecaster,
    series=series_dict,
    X_train = X_train,
    y_train= y_train,
    X_train_encoding = X_train_encoding,
    X_test = X_test,
    y_test = y_test,
    X_test_encoding = X_test_encoding,
    levels = ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
    metrics=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric = True
)
display(metrics_one_step_ahead)


metrics == metrics_one_step_ahead

NameError: name 'series_dict' is not defined

In [30]:
id = 'id_1003'
y_pred = predictions[id]
y_true = series_dict[id].loc[y_pred.index]
y_train = series_dict[id].loc[:end_train].iloc[forecaster.window_size:]
print(y_true.sum())
print(y_pred.sum())
print(y_train.sum())
print(y_true.diff().sum())
print(y_pred.diff().sum())
print(y_train.diff().sum())


mean_absolute_scaled_error(
    y_true=y_true,
    y_pred=y_pred,
    y_train=y_train
)

161627.84101104736
211887.69980665128
293839.05683517456
-1214.7492065429688
-1686.317492489301
2149.249496459961


2.6562753666913723

In [31]:
print(y_true)
print(y_pred)
print(y_train)

2016-08-01    2618.750099
2016-08-02    1972.250294
2016-08-03    1866.870991
2016-08-04    1693.000397
2016-08-05    1643.130501
                 ...     
2016-12-27    2058.000000
2016-12-28    1999.250000
2016-12-29    1942.250000
2016-12-30    1939.250000
2016-12-31    1964.500000
Freq: D, Name: id_1003, Length: 153, dtype: float64
2016-08-01    3174.668788
2016-08-02    2716.991572
2016-08-03    2514.637226
2016-08-04    2514.637226
2016-08-05    2414.253784
                 ...     
2016-12-27    2611.095911
2016-12-28    2655.210831
2016-12-29    2548.936148
2016-12-30    2611.095911
2016-12-31    2611.095911
Freq: D, Name: id_1003, Length: 153, dtype: float64
timestamp
2016-01-02    1750.000198
2016-01-03    1455.750492
2016-01-04    2141.000198
2016-01-05    2384.870697
2016-01-06    2357.750496
                 ...     
2016-07-27    1830.871780
2016-07-28    1776.250992
2016-07-29    1644.001091
2016-07-30    1943.500587
2016-07-31    3658.369995
Freq: D, Name: id_1003, Leng

In [23]:
for k, v in series_dict_train.items():
    print(k)
    print(len(v))
    print(v.isna().sum())

id_1000
213
0
id_1001
30
0
id_1002
183
2
id_1003
213
73
id_1004
91
0


## backtesting vs _predict_and_calculate_metrics_multiseries_one_step_ahead

In [13]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [4]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
series = fetch_dataset(name="items_sales")
exog = pd.DataFrame(
    {
        'day_of_week': series.index.dayofweek
    },
    index = series.index
)
initial_train_size = 927

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),
    ForecasterAutoregMultiVariate(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        steps              = 1,
        level              = 'item_1',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
    )
]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series,
        exog=exog,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series,
            exog               = exog,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)
backtest_predictions [              item_1     item_2     item_3
2014-07-16  25.54998  11.828119  10.950973,               item_1     item_2     item_3
2014-07-17  26.43482  11.461392  10.137645,                item_1     item_2     item_3
2014-07-18  25.169748  10.101028  13.301167,                item_1     item_2     item_3
2014-07-19  24.260504  10.195987  14.719568,                item_1    item_2    item_3
2014-07-20  22.369294  10.74458  9.404583,                item_1     item_2     item_3
2014-07-21  23.188304  11.028047  12.398843,                item_1     item_2     item_3
2014-07-22  25.296614  10.363722  11.614964,                item_1     item_2     item_3
2014-07-23  27.038102  10.997707  11.274721,                item_1    item_2     item_3
2014-07-24  25.916304  8.070517  10.566657,                item_1    item_2    item_3
2014-07-25  26.

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


mean_absolute_scaled_error one-step-ahead
[903, 903, 903]


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.825973,0.041122,0.541134
1,item_2,2.251201,0.146863,0.955679
2,item_3,3.187688,0.216736,0.860802
3,average,2.088287,0.134907,0.785872
4,weighted_average,2.088287,0.134907,0.785872
5,pooling,2.088287,0.134907,0.825939


backtest_predictions [               item_1
2014-07-16  26.211801,                item_1
2014-07-17  26.142183,                item_1
2014-07-18  25.235666,                item_1
2014-07-19  24.006873,                item_1
2014-07-20  21.338697,                item_1
2014-07-21  22.517451,                item_1
2014-07-22  24.960068,                item_1
2014-07-23  26.520083,                item_1
2014-07-24  26.416822,                item_1
2014-07-25  26.560177,                item_1
2014-07-26  24.492337,                item_1
2014-07-27  21.746483,                item_1
2014-07-28  22.704891,               item_1
2014-07-29  24.62009,                item_1
2014-07-30  24.825964,                item_1
2014-07-31  24.502625,                item_1
2014-08-01  24.706683,                item_1
2014-08-02  23.411667,                item_1
2014-08-03  22.537342,                item_1
2014-08-04  23.191749,                item_1
2014-08-05  25.444608,                item_1
2014-08-06  2

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.794891,0.039537,0.520771


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.794891,0.039537,0.520771


In [5]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),
    ForecasterAutoregMultiVariate(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        steps              = 1,
        level              = 'item_1',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
    )
]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series_dict,
        exog=exog_dict,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series_dict,
            exog               = exog_dict,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series_dict,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:836: MissingValuesWarning: NaNs detected in `y_train`. They have been dropped because the target variable cannot have NaN values. Same rows have been dropped from `X_train` to maintain alignment. This is caused by series with interspersed NaNs. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:858: MissingValuesWarning: NaNs detected in `X_train`. Some regressors do not allow NaN values during training. If you want to drop them, set `forecaster.dropna_from_series = True`. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(


ValueError: No objects to concatenate

In [8]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecaster = ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 10,
        encoding           = 'ordinal',
        # transformer_series = StandardScaler(),
        # transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    )


metrics_backtesting, predictions = backtesting_forecaster_multiseries(
    series=series_dict,
    exog=exog_dict,
    forecaster=forecaster,
    steps=10,
    metric=metrics,
    initial_train_size = initial_train_size,
    refit=False,
    levels=['item_1', 'item_2', 'item_3'],
    add_aggregated_metric=False,
    show_progress=False
)

display(metrics_backtesting)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:836: MissingValuesWarning: NaNs detected in `y_train`. They have been dropped because the target variable cannot have NaN values. Same rows have been dropped from `X_train` to maintain alignment. This is caused by series with interspersed NaNs. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(
/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:858: MissingValuesWarning: NaNs detected in `X_train`. Some regressors do not allow NaN values during training. If you want to drop them, set `forecaster.dropna_from_series = True`. 
 You can suppress this warning using: warnings.simplefilter('ignore', category=MissingValuesWarning)
  warnings.warn(


AAAAAAAAAAAAAAAAA


ValueError: No objects to concatenate